In [1]:
import pandas as pd

import os
dataset_path = "/Users/bruce/Downloads/datasets/so/pastThreeMonth"

filenames = list(os.walk(dataset_path))[0][2]

Q_filenames = [name for name in filenames if name[0]=="Q"]

A_filenames = [name for name in filenames if name[0]=="A"]

### 读取并拆分数据

In [2]:
q_df = None
for i in range(len(Q_filenames)):
    qfile_path = os.path.join(dataset_path, Q_filenames[i])
    if i==0:
        q_df = pd.read_csv(qfile_path)
    else:
        tmp_df = pd.read_csv(qfile_path)
        q_df = pd.concat([q_df, tmp_df])

q_df.drop_duplicates(subset=['Id'], keep='first', inplace=True)

/Users/bruce/opt/anaconda3/envs/py38_torch/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (10,21) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
a_df = None
for i in range(len(A_filenames)):
    afile_path = os.path.join(dataset_path, A_filenames[i])
    if i==0:
        a_df = pd.read_csv(afile_path)
    else:
        tmp_df = pd.read_csv(afile_path)
        a_df = pd.concat([a_df, tmp_df])

a_df.drop_duplicates(subset=['Id'], keep='first', inplace=True)

/Users/bruce/opt/anaconda3/envs/py38_torch/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/bruce/opt/anaconda3/envs/py38_torch/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (10,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/bruce/opt/anaconda3/envs/py38_torch/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (10,12,21) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
# 数据中前23列是post，23列之后是user
post_columns = a_df.columns[:23]
user_columns = a_df.columns[23:]

In [5]:
# 将question、answer、user分开
u_df = pd.concat([a_df[user_columns], q_df[user_columns]])
user_columns = [col for col in user_columns]
user_columns[0] = 'Id'
user_columns[2] = 'CreationDate'
u_df.columns = user_columns
a_df = a_df[post_columns]
q_df = q_df[post_columns]

In [6]:
# 注意，因为数据集的原因， 因为有些问题的答案为空，有可能问题的回答时间在数据集时间范围之外; 有些答案的parentId不在问题id中
# 因此要找到a_df和q_df里共有的问题id
common_id = pd.merge(left=a_df['ParentId'], right = q_df['Id'], left_on='ParentId', right_on='Id', how='inner')

In [7]:
common_id.drop_duplicates(subset=['ParentId'], keep='first', inplace=True)

In [8]:
a_df = a_df.loc[a_df['ParentId'].isin(common_id['ParentId']), :]
q_df = q_df.loc[q_df['Id'].isin(common_id['Id']), :]

### 1. 获取label

330350

In [10]:
# 使用排序来得到分数越大，时间越早的答案
a_df['-Score'] = a_df['Score']*(-1)
sort_df = a_df.sort_values(by=['ParentId', '-Score', 'CreationDate']).loc[:,['ParentId','Score', 'CreationDate', 'Id']]

In [11]:
# 然后使用df去重，仅保留第一条
sort_df.drop_duplicates(subset=['ParentId'], keep='first', inplace=True)

In [12]:
a_df.drop(columns=['-Score'], inplace=True)

In [13]:
# 拼接
qtime_df = q_df.loc[:, ['Id', 'CreationDate']]
sort_df = pd.merge(left=sort_df, right=qtime_df, left_on='ParentId', right_on = 'Id', how='inner')

In [14]:
# 设置列名
sort_df.columns = ['ParentId', 'a_Score', 'a_CreationDate', 'a_Id', 'q_Id', 'q_CreationDate']

In [15]:
import time
# 将字符串格式时间转换为时间戳
def str_to_timestep(s):
    timeArray = time.strptime(s.split('.')[0], "%Y-%m-%d %H:%M:%S")
    timeStamp = int(time.mktime(timeArray))
    return timeStamp

In [16]:
sort_df['a_CreationDateStamp'] = sort_df['a_CreationDate'].apply(str_to_timestep)
sort_df['q_CreationDateStamp'] = sort_df['q_CreationDate'].apply(str_to_timestep)
# 计算回答和提问的相隔时间，单位是s
sort_df['TimeDelta'] = sort_df['a_CreationDateStamp'] - sort_df['q_CreationDateStamp']

#### <font color='blue'>获得了label，包含问题的id，答案的id， 时间间隔</font>

In [17]:
label_df = sort_df.loc[:, ['q_Id', 'a_Id', 'TimeDelta']]
del sort_df

In [18]:
label_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 327349 entries, 0 to 327348
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype
---  ------     --------------   -----
 0   q_Id       327349 non-null  int64
 1   a_Id       327349 non-null  int64
 2   TimeDelta  327349 non-null  int64
dtypes: int64(3)
memory usage: 10.0 MB


### 2. 存储label、user、answer、question

In [31]:
output_path = os.path.join(dataset_path, 'output')

In [ ]:
output_filepath = os.path.join(output_path, 'label.csv')
label_df.to_csv(output_filepath)

In [ ]:
user_filepath = os.path.join(output_path, 'user.csv')
ans_filepath = os.path.join(output_path, 'answer.csv')
ques_filepath = os.path.join(output_path, 'question.csv')

u_df.to_csv(user_filepath)
a_df.to_csv(ans_filepath)
q_df.to_csv(ques_filepath)

### 3. 计算基于Tag的特征 

In [19]:
def handle_tag(tag):
    # 去除尖括号
    return tag.replace("<", " ").replace(">", " ").strip()

In [20]:
# 因为数据原因，部分数据没有label，因此将将没有label的数据剔除
q_df = label_df.merge(how="left", right = q_df, left_on='q_Id', right_on='Id')

q_df.drop(columns=['q_Id', 'a_Id', 'TimeDelta'], inplace=True)

In [21]:
# 处理tag
q_df["Tags"] = q_df["Tags"].apply(handle_tag)
q_df['Tags'] = q_df['Tags'].apply(lambda x: x.split())

In [22]:
# tag和question之间的关系
from tqdm import trange

tag_questions = []
q_num = q_df.shape[0]
for i in trange(q_num):
    q_id = q_df.iloc[i]['Id']
    tags = q_df.iloc[i]['Tags']
    for t in tags:
        tag_questions.append([t, q_id])

100%|██████████| 327349/327349 [02:14<00:00, 2434.56it/s]


#### <font color='blue'>(1)获得tag的流行度</font>

tag和question对应的dataframe

In [23]:
tag_ques_df = pd.DataFrame(tag_questions)

tag_ques_df.columns=['Tag', 'q_Id']

tag_features_df = tag_ques_df.groupby('Tag').size().reset_index(name='QuestionsNumber').sort_values(by=['Tag'])

跟label拼接，得到tag、question、time对应关系

In [24]:
tag_ques_time_df = pd.merge(left = tag_ques_df, right = label_df,left_on=tag_ques_df.q_Id, right_on=label_df.q_Id)

In [25]:
threshold = 60*60  # 一小时
tag_ques_time_df['WhetherQuick60'] = tag_ques_time_df['TimeDelta']<threshold
tag_ques_time_df['WhetherQuick60'] = tag_ques_time_df['WhetherQuick60'].astype('float')

In [26]:
threshold2 = 16*60  # 16分钟
tag_ques_time_df['WhetherQuick16'] = tag_ques_time_df['TimeDelta']<threshold2
tag_ques_time_df['WhetherQuick16'] = tag_ques_time_df['WhetherQuick16'].astype('float')

In [27]:
# 每个tag下平均响应时间
tag_time_df = tag_ques_time_df.groupby('Tag')['TimeDelta'].mean().reset_index(name='AverageTime'\
                                                                             ).sort_values(by=['Tag'])
# 每个tag下得到快速响应的问题个数(分为16分钟和60分钟)
tag_quick16_df = tag_ques_time_df.groupby('Tag')['WhetherQuick16'].sum().reset_index(name='QuickNumber16'\
                                                                                ).sort_values(by=['Tag'])
tag_quick60_df = tag_ques_time_df.groupby('Tag')['WhetherQuick60'].sum().reset_index(name='QuickNumber60'\
                                                                                ).sort_values(by=['Tag'])

#### <font color='blue'>(2)获得tag的平均响应时间、快速个数</font>

In [28]:
tag_features_df['AverageTime'] = tag_time_df['AverageTime'].astype(int)
tag_features_df['QuickNumber16'] = tag_quick16_df['QuickNumber16'].astype(int)
tag_features_df['QuickNumber60'] = tag_quick60_df['QuickNumber60'].astype(int)

In [29]:
del tag_time_df;
del tag_quick16_df;
del tag_quick60_df;

In [ ]:
tag_features_df.info()

In [ ]:
# tag_features_df['QuickNumber']=tag_features_df['QuickNumber'].fillna(0)
# tag_features_df['AverageTime']=tag_features_df['AverageTime'].fillna(tag_features_df['AverageTime'].median())

#### (3)存储tag具有的属性

In [ ]:
tag_features_filepath = os.path.join(output_path, 'tag_feature.csv')
tag_features_df.to_csv(tag_features_filepath)

#### (4) 通过tag属性来计算question的tag-based属性 

In [32]:
tmp_df = pd.merge(left = tag_ques_df, right = tag_features_df, left_on = 'Tag', right_on = 'Tag')

In [33]:
tag_popularity = tmp_df.groupby('q_Id')['QuestionsNumber'\
                                       ].sum().reset_index(name='TagPopularity').sort_values(by=['q_Id'])

In [34]:
tag_averageTime = tmp_df.groupby('q_Id')['AverageTime'\
                                        ].mean().reset_index(name='TagAverageTime').sort_values(by=['q_Id'])

In [35]:
tag_averageQuick16 = tmp_df.groupby('q_Id')['QuickNumber16'\
                                        ].mean().reset_index(name='TagAverageQuick').sort_values(by=['q_Id'])
tag_averageQuick60 = tmp_df.groupby('q_Id')['QuickNumber60'\
                                        ].mean().reset_index(name='TagAverageQuick').sort_values(by=['q_Id'])

In [36]:
tag_based_features = tag_popularity
tag_based_features['TagAverageTime'] = tag_averageTime['TagAverageTime']
tag_based_features['TagAverageQuick16'] = tag_averageQuick16['TagAverageQuick']
tag_based_features['TagAverageQuick60'] = tag_averageQuick60['TagAverageQuick']

#### (5)存储tag-based的属性 

In [ ]:
tag_based_features_filepath = os.path.join(output_path, 'tag_based_feature.csv')
tag_based_features.to_csv(tag_based_features_filepath)

### 3. 计算不基于Tag的特征, 即问题本身的特征

In [37]:
self_features = pd.DataFrame()

In [38]:
self_features['Id'] = q_df['Id']

In [39]:
self_features['num_code_snippet'] = 0
self_features['code_len'] = 0
self_features['num_image'] = 0
self_features['body_len'] = 0
self_features['title_len'] = 0
self_features['end_que_mark'] = 0
self_features['begin_que_word'] = 0
self_features['is_weekend'] = 0

#### (1) 通过问题的Body、Title、CreationDate来获得question自身的特征

In [40]:
import re
from datetime import datetime

code_pattern = re.compile(r"<code>(.*?)</code>", flags=re.DOTALL)
img_pattern = re.compile(r"<img(.*?)>", flags=re.DOTALL)

def get_num_code_snippet(s):
    res = code_pattern.findall(s)
    return len(res)

def get_code_len(s):
    res = code_pattern.findall(s)
    code_len = sum([len(code) for code in res])
    return code_len

def get_num_img(s):
    res = img_pattern.findall(s)
    return len(res)

def get_body_len(s):
    return len(s)

def get_title_len(s):
    return len(s)

def get_end_que_mark(s):
    s = s.strip()
    return len(s)!=0 and s[-1] == '?'

def get_begin_que_word(s):
    s = s.strip().lower()
    return len(s)!=0 and (s.startswith('wh') or s.startswith('how'))

def get_is_weekend(s):
    date_time_obj = datetime.strptime(s, '%Y-%m-%d %H:%M:%S')
    return date_time_obj.weekday()>=5    # 周一是0，周二是1，... , 周日是6

In [41]:
from tqdm import tqdm

tqdm.pandas()

self_features['num_code_snippet'] = q_df['Body'].progress_apply(get_num_code_snippet)
self_features['code_len'] = q_df['Body'].progress_apply(get_code_len)
self_features['num_image'] = q_df['Body'].progress_apply(get_num_img)
self_features['body_len'] = q_df['Body'].progress_apply(get_body_len)
self_features['title_len'] = q_df['Title'].progress_apply(get_title_len)
self_features['end_que_mark'] = q_df['Title'].progress_apply(get_end_que_mark)
self_features['begin_que_word'] = q_df['Title'].progress_apply(get_begin_que_word)
self_features['is_weekend'] = q_df['CreationDate'].progress_apply(get_is_weekend)

/Users/bruce/opt/anaconda3/envs/py38_torch/lib/python3.8/site-packages/tqdm/std.py:670: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|██████████| 327349/327349 [00:05<00:00, 62042.74it/s]


#### (2) 存储问题自身特征

In [ ]:
self_features_filepath = os.path.join(output_path, 'self_feature.csv')
self_features.to_csv(self_features_filepath)

#### (3) 将问题self和tag based的特征合并起来进行存储

In [42]:
self_and_tag_features = pd.merge(left=self_features, right=tag_based_features, left_on='Id', right_on='q_Id')

In [43]:
self_and_tag_features.drop(columns=['q_Id'], inplace=True)

In [44]:
self_and_tag_features['end_que_mark'] = self_and_tag_features['end_que_mark'].astype(int)
self_and_tag_features['begin_que_word'] = self_and_tag_features['begin_que_word'].astype(int)
self_and_tag_features['is_weekend'] = self_and_tag_features['is_weekend'].astype(int)

In [ ]:
features_df = self_and_tag_features
features_filepath = os.path.join(output_path, 'feature.csv')
features_df.to_csv(features_filepath)

## 最后提取得到features和label

In [ ]:
(label_df["q_Id"].sort_values() == features_df["Id"].sort_values()).sum()

In [ ]:
data = pd.merge(left=features_df, right=label_df, left_on='Id', right_on='q_Id')

In [ ]:
data.drop(columns=['q_Id', 'a_Id'], inplace=True)

In [ ]:
data_filepath = os.path.join(output_path, 'data.csv')
data.to_csv(data_filepath)